## Assignment 2 -- Salvador GARCIA -- s1655274

## Problem 1

In [1]:
srand(1234)
n = 1000
d = 10
B = randn(d, n)
alpha = zeros(d,1)
c = rand(d,1)
for i = 1:d
    alpha[i] = n/i
    c[i] = 1/(sqrt(alpha[i]))
end
gamma = 1/d
tau = 1
function ComputeESO(B,alpha,gamma,tau)
    n = size(B, 2)
    d = size(B, 1)
    for j = 1:d
        alpha[j] = n/j
        B[j,:] = B[j,:]/sqrt(alpha[j])
    end
    M = B'* B + 2 *gamma* eye(n)
    A = chol(M)

    omega = zeros(d,1)
    v = zeros(n,1)
    for i = 1:n
        for j = 1:d
            if A[j,i]!=0
                omega[j] += 1
            end
        end
    end
    
    for i = 1:n
        for j = 1:d
            v[i] += v[i]+(1+(tau-1)*(omega[j]-1)/(n-1))*A[j,i]*A[j,i]
        end
    end
    return v
end
v = ComputeESO(B, alpha, gamma, tau)

1000×1 Array{Float64,2}:
 126.456   
  61.9053  
  35.5415  
  16.1533  
   7.64681 
   5.48392 
   2.27295 
   1.55544 
   1.15212 
   0.813018
   0.925661
   0.480439
   0.168373
   ⋮       
   0.188166
   1.85226 
   0.93004 
   0.621652
   2.02083 
   0.683703
   0.91749 
   0.809517
   2.72723 
   0.318202
   0.934744
   0.315108

In [2]:
B

10×1000 Array{Float64,2}:
  0.0274279  -0.0177246     0.0494633   …   0.0190564  -0.0174129 
 -0.0403272  -0.000862755  -0.0624639       0.0160219   0.0159993 
 -0.0270837   0.00701438    0.0605506       0.0690947  -0.0346566 
 -0.0571053   0.11718      -0.0699957      -0.0523323  -0.0576553 
  0.0611224  -0.0585317    -0.227077       -0.0659594  -0.0200894 
  0.171331    0.00852801   -0.00573314  …   0.0261033  -0.0271092 
  0.0445784  -0.0210149     0.0126315      -0.0118249   0.0800076 
 -0.0243048   0.0330682     0.0688063      -0.139951    0.103153  
  0.0476556   0.00684156   -0.0294237       0.0516403   0.179206  
 -0.0516984  -0.150343     -0.0602707       0.143553    0.00995793

In [3]:
using Distributions

x = rand(n,1)
fx = 1/2*x'*B'*B*x + c'*B*x + 1/2*c'*c + gamma*x'x
dfx = x'*B'*B + c'*B + 2*gamma*x'

p = zeros(n,1)
p = p.+tau/n

h = -diagm(1./v[1:n])*dfx'

f1 = zeros(n, 1)
f2 = zeros(n, 1)
for i = 1:n
    f1[i] = p[i]*dfx[i]*v[i]
    f2[i] = p[i]*h[i]*v[i]*h[i]
end

RHS = (fx +sum(f1)+sum(f2))[1]
x_samples_vec = zeros(1000, 1)
for i = 1:1000
    x_samples = copy(x)
    ss = sample(1:n, tau, replace = false)
    for j = 1:tau
        x_samples[ss[j]] = x_samples[ss[j]] + h[ss[j]]*v[ss[j]]
    end
    x_samples_vec[i] = (1/2*x_samples'*B'*B*x_samples + c'*B*x_samples + 1/2*c'*c + gamma*x_samples'x_samples)[1]
    #println("original: $fx ;;;; nuevo: $x_samples_vec")
end
LHS = mean(x_samples_vec)
println("LHS:  $LHS ;;;; RHS:  $RHS")

LHS:  53.30413815181462 ;;;; RHS:  54.2265140205425


In [4]:
using Distributions
    x = zeros(n,1)
    p = zeros(n,1)
    p = p.+tau/n

for i = 1:10
    fx = 1/2*x'*B'*B*x + c'*B*x + 1/2*c'*c + gamma*x'x
    println("Objetivo: $fx")
    dfx = x'*B'*B + c'*B + 2*gamma*x'
    h = -diagm(1./v[1:n])*dfx'
    ss = sample(1:n, tau, replace = false)
    for j = 1:tau
        x[ss[j]] = x[ss[j]] + h[ss[j]]*v[ss[j]]
    end
end



Objetivo: [0.0275]
Objetivo: [0.0269078]
Objetivo: [0.026822]
Objetivo: [0.0260549]
Objetivo: [0.0257862]
Objetivo: [0.025783]
Objetivo: [0.025203]
Objetivo: [0.0247457]
Objetivo: [0.0247349]
Objetivo: [0.0246872]


In [7]:
using Distributions


salida = 0
for k = 1:100
    x = rand(n,1)
    fx = 1/2*x'*B'*B*x + c'*B*x + 1/2*c'*c + gamma*x'x
    dfx = x'*B'*B + c'*B + 2*gamma*x'

    p = zeros(n,1)
    p = p.+tau/n

    h = -diagm(1./v[1:n])*dfx'

    f1 = zeros(n, 1)
    f2 = zeros(n, 1)
    for i = 1:n
        f1[i] = p[i]*dfx[i]*v[i]
        f2[i] = p[i]*h[i]*v[i]*h[i]
    end

    RHS = fx +sum(f1)+sum(f2)

    x_samples_vec = zeros(100,1)

    for i = 1:1000
        x_samples = copy(x)
        ss = sample(1:n, tau, replace = false)
        for j = 1:tau
            x_samples[ss[j]] = x_samples[ss[j]] + h[ss[j]]*v[ss[j]]
        end
        x_samples_vec = (1/2*x_samples'*B'*B*x_samples + c'*B*x_samples + 1/2*c'*c + gamma*x_samples'x_samples)[1]
    end
    LHS = mean(x_samples_vec)
    
    if (RHS[1]-LHS)>=0
        salida = salida +1
    end
end
println(tau)
print(salida/100)



1
1.0